## ABC applied to position reconstruction

*Bart Pelssers, 26-02-2018*

This notebook provides some ingredients for applying the ABC algorithm to position reconstruction, the most basic case. Just to test the framework.


* Provides:
  * prior mean
  * forward model
  * summary statistic

In [1]:
import numpy as np

In [2]:
# Using the XENON1T data processor
from pax import utils
from pax.configuration import load_configuration
from pax.PatternFitter import PatternFitter
from pax.plugins.io.WaveformSimulator import uniform_circle_rv

In [3]:
class PriorPosition():
    """Implements the calculation of the mean of a prior
       given a pattern (either from data or from the forward model)
    """
    
    def __init__(self):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        
        # PMT positions
        pmt_config = config['DEFAULT']['pmts']
        
        # List of dicts {'x': , 'y'}, which the position
        self.positions = [pmt['position'] for pmt in pmt_config][:127]

    def __call__(self, pattern):
        # The id of the PMT that sees most light
        max_pmt = np.argmax(pattern)
        
        # The position of that PMT
        pos = self.positions[max_pmt]
        
        return pos['x'], pos['y']        
        

class Model():
    """Implements the forward model for ABC project.
    
       The forward model used here is the most basic test case.
       It draws from the per-PMT S2 LCE maps to provide a
       hitpattern for a given x,y. Assumes all top PMTs live.
       Also the total number of detected photo-electrons needs to
       be specified, this is set constant by default.
    """

    def __init__(self):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        
        # The per-PMT S2 LCE maps (and zoom factor which is a technical detail)
        lce_maps = config['WaveformSimulator']['s2_patterns_file']
        lce_map_zoom = config['WaveformSimulator']['s2_patterns_zoom_factor']

        # Simulate the right PMT response
        qes = np.array(config['DEFAULT']['quantum_efficiencies'])
        top_pmts = config['DEFAULT']['channels_top']
        errors = config['DEFAULT']['relative_qe_error'] + config['DEFAULT']['relative_gain_error']

        # Set up the PatternFitter which sample the LCE maps
        self.pf = PatternFitter(filename=utils.data_file_name(lce_maps),
                                zoom_factor=lce_map_zoom,
                                adjust_to_qe=qes[top_pmts],
                                default_errors=errors)

    def __call__(self, x, y, n_obs = 500):
        """Returns a hitpattern of n_obs photo-electrons
           for given x, y position.
        """
        
        return n_obs * self.pf.expected_pattern((x, y))


class Generator():
    """Generates test hitpatterns by drawing from LCE maps (forward model)"""
    
    def __init__(self, model):
        # Get some settings from the XENON1T detector configuration
        config = load_configuration('XENON1T')
        self.tpc_radius = config['DEFAULT']['tpc_radius']
        
        # Use the forward model also as generator for now
        self.model = model
    
    def __call__(self):
        return self.model(*uniform_circle_rv(self.tpc_radius))

In [4]:
# Setup the Models
model = Model()
prior_mean = PriorPosition()
generator = Generator(model)

In [5]:
# Example pattern from some unknown x,y position
generator()

array([   0.39210056,    0.42116387,    0.49146609,    0.6492735 ,
          0.78701824,    1.04278539,    1.56752916,    3.02345386,
          8.8167784 ,   30.84459043,   48.84053092,   20.50164816,
          5.88254313,    2.50329281,    1.44128656,    0.89463838,
          0.67413846,    0.63135954,    0.46844248,    0.46262775,
          0.46326254,    0.39417298,    0.38223789,    0.39357651,
          0.39175567,    0.34310544,    0.3747821 ,    0.38408381,
          0.31252028,    0.38427949,    0.33913843,    0.42675795,
          0.42199762,    0.44085829,    0.45643749,    0.52858694,
          0.70938193,    0.8185129 ,    0.80851395,    0.96576379,
          1.38576771,    2.31692014,    5.21790655,   26.02008392,
        106.69719651,   52.66369507,    8.90867158,    2.79064265,
          1.49508207,    1.21232133,    1.08814447,    0.89953175,
          0.69876625,    0.67414192,    0.68006458,    0.71851844,
          0.56563009,    0.64645594,    0.62075072,    0.56685

In [6]:
# Example pattern from x=24, y=-12

# The range of x and y is [-47.884375 cm, -47.884375 cm]
# But x**2 + y**2 < 47.884375**2
# Otherwise m() will raise and exception

pattern = model(24, -12)

print("Length of pattern: %d, Sum of pattern: %.2f" % (len(pattern), pattern.sum()))
print(pattern)

Length of pattern: 127, Sum of pattern: 500.00
[  0.38380166   0.41537353   0.39553333   0.50098828   0.52944289
   0.58591603   0.56589008   0.70695663   0.95604715   1.24240681
   1.73795135   2.58792553   3.52043052   4.09192643   4.30745586
   2.80196583   2.12977881   1.44824766   0.94742216   0.75621597
   0.69163486   0.60090247   0.6541341    0.46846789   0.42496789
   0.44825287   0.46466767   0.45177168   0.3799691    0.34540879
   0.46125516   0.36648025   0.35792033   0.37139642   0.44512946
   0.4700616    0.65978395   0.70655844   0.72618991   0.86043664
   1.02138069   0.91583681   1.10452859   1.78946327   2.40828215
   4.67516569   9.22073926  11.85984413   8.55169094   4.2165689
   2.39809303   1.51885938   1.1952463    0.99576857   0.99345914
   0.71552314   0.75153697   0.7784726    0.66604251   0.62908966
   0.61447725   0.7072028    0.60351863   0.57881621   0.83823376
   0.68045143   0.75001609   0.71080193   0.92486869   1.12672182
   1.14216264   1.68770307   3

In [7]:
# If using a 2D Normal prior this would be a good guess for the mean (x,y) of that prior.
prior_mean(pattern)

(19.536776256292445, -13.679798006972462)